In [1]:
from langchain.agents import create_sql_agent
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_types import AgentType
from langchain_openai.chat_models import ChatOpenAI

In [2]:
db = SQLDatabase.from_uri("sqlite:///./data/demo.db")
toolkit = SQLDatabaseToolkit(db=db, llm=ChatOpenAI(temperature=0))

In [3]:
agent_executor = create_sql_agent(
    llm=ChatOpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [4]:
agent_executor.invoke("Identify all of the tables")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`


Orders, Products, Users
Invoking: `sql_db_schema` with `{'table_names': 'Orders, Products, Users'}`



CREATE TABLE "Orders" (
	"OrderID" INTEGER, 
	"UserID" INTEGER, 
	"ProductID" INTEGER, 
	"Quantity" INTEGER NOT NULL, 
	"OrderDate" DATE NOT NULL, 
	PRIMARY KEY ("OrderID"), 
	FOREIGN KEY("UserID") REFERENCES "Users" ("UserID"), 
	FOREIGN KEY("ProductID") REFERENCES "Products" ("ProductID")
)

/*
3 rows from Orders table:
OrderID	UserID	ProductID	Quantity	OrderDate
1	1	1	1	2023-05-01
2	2	2	3	2023-05-05
3	3	3	2	2023-05-07
*/


CREATE TABLE "Products" (
	"ProductID" INTEGER, 
	"ProductName" TEXT NOT NULL, 
	"Price" REAL NOT NULL, 
	"StockQuantity" INTEGER NOT NULL, 
	PRIMARY KEY ("ProductID")
)

/*
3 rows from Products table:
ProductID	ProductName	Price	StockQuantity
1	Laptop	1000.0	20
2	Headphones	50.0	100
3	Mouse	25.0	150
*/


CREATE TABLE "Users" (
	"UserID" INTEGER, 
	"FirstName"

{'input': 'Identify all of the tables',
 'output': 'The database contains the following tables:\n1. Orders\n2. Products\n3. Users'}

In [5]:
user_sql = agent_executor.invoke(
    "Add 5 new users to the database. Their names are: John, Mary, Peter, Paul, and Jane. Run the following SQL query against the database and add the users."
)
print(user_sql)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`


Orders, Products, Users
Invoking: `sql_db_schema` with `Users`



CREATE TABLE "Users" (
	"UserID" INTEGER, 
	"FirstName" TEXT NOT NULL, 
	"LastName" TEXT NOT NULL, 
	"Email" TEXT NOT NULL, 
	"DateJoined" DATE NOT NULL, 
	PRIMARY KEY ("UserID"), 
	UNIQUE ("Email")
)

/*
3 rows from Users table:
UserID	FirstName	LastName	Email	DateJoined
1	Alice	Smith	alice.smith@email.com	2023-01-01
2	Bob	Johnson	bob.johnson@email.com	2023-02-15
3	Charlie	Brown	charlie.brown@email.com	2023-04-10
*/Based on the schema of the "Users" table, I can see that the relevant columns for adding new users are "FirstName", "LastName", "Email", and "DateJoined". I will now run the SQL query to add the new users.

```sql
INSERT INTO Users (FirstName, LastName, Email, DateJoined)
VALUES ('John', 'Doe', 'john.doe@example.com', DATE('now')),
       ('Mary', 'Smith', 'mary.smith@example.com', DATE('now')),
       ('Peter', 'Johnson', 'pe

In [6]:
user_sql

{'input': 'Add 5 new users to the database. Their names are: John, Mary, Peter, Paul, and Jane. Run the following SQL query against the database and add the users.',
 'output': 'Based on the schema of the "Users" table, I can see that the relevant columns for adding new users are "FirstName", "LastName", "Email", and "DateJoined". I will now run the SQL query to add the new users.\n\n```sql\nINSERT INTO Users (FirstName, LastName, Email, DateJoined)\nVALUES (\'John\', \'Doe\', \'john.doe@example.com\', DATE(\'now\')),\n       (\'Mary\', \'Smith\', \'mary.smith@example.com\', DATE(\'now\')),\n       (\'Peter\', \'Johnson\', \'peter.johnson@example.com\', DATE(\'now\')),\n       (\'Paul\', \'Brown\', \'paul.brown@example.com\', DATE(\'now\')),\n       (\'Jane\', \'Davis\', \'jane.davis@example.com\', DATE(\'now\'));\n```\n\nThis query will add 5 new users to the database with the specified names and email addresses.'}

In [7]:
SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
If the question does not seem related to the database, just return "I don't know" as the answer.
"""

In [8]:
# Updating the agent_executor to include DML statements:
agent_executor = create_sql_agent(
    llm=ChatOpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    prefix=SQL_PREFIX,
)

In [9]:
agent_executor.invoke(user_sql)



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with ``


Orders, Products, Users
Invoking: `sql_db_schema` with `Users`



CREATE TABLE "Users" (
	"UserID" INTEGER, 
	"FirstName" TEXT NOT NULL, 
	"LastName" TEXT NOT NULL, 
	"Email" TEXT NOT NULL, 
	"DateJoined" DATE NOT NULL, 
	PRIMARY KEY ("UserID"), 
	UNIQUE ("Email")
)

/*
3 rows from Users table:
UserID	FirstName	LastName	Email	DateJoined
1	Alice	Smith	alice.smith@email.com	2023-01-01
2	Bob	Johnson	bob.johnson@email.com	2023-02-15
3	Charlie	Brown	charlie.brown@email.com	2023-04-10
*/Based on the schema of the "Users" table, I can see that the relevant columns for adding new users are "FirstName", "LastName", "Email", and "DateJoined". 

To add the 5 new users to the database, I will run the following SQL query:

```sql
INSERT INTO Users (FirstName, LastName, Email, DateJoined)
VALUES
  ('John', 'Doe', 'john.doe@example.com', '2023-05-01'),
  ('Mary', 'Smith', 'mary.smith@example.com', '2023-05-02'),
  ('Pet

{'input': 'Add 5 new users to the database. Their names are: John, Mary, Peter, Paul, and Jane. Run the following SQL query against the database and add the users.',
 'output': 'Based on the schema of the "Users" table, I can see that the relevant columns for adding new users are "FirstName", "LastName", "Email", and "DateJoined". \n\nTo add the 5 new users to the database, I will run the following SQL query:\n\n```sql\nINSERT INTO Users (FirstName, LastName, Email, DateJoined)\nVALUES\n  (\'John\', \'Doe\', \'john.doe@example.com\', \'2023-05-01\'),\n  (\'Mary\', \'Smith\', \'mary.smith@example.com\', \'2023-05-02\'),\n  (\'Peter\', \'Johnson\', \'peter.johnson@example.com\', \'2023-05-03\'),\n  (\'Paul\', \'Brown\', \'paul.brown@example.com\', \'2023-05-04\'),\n  (\'Jane\', \'Davis\', \'jane.davis@example.com\', \'2023-05-05\');\n```\n\nPlease note that I have assumed the format of the "DateJoined" column to be \'YYYY-MM-DD\'.'}

In [10]:
agent_executor.invoke("Do we have a Peter in the database?")



> Entering new AgentExecutor chain...

Invoking: `sql_db_list_tables` with `{}`


Orders, Products, Users
Invoking: `sql_db_schema` with `Users`



CREATE TABLE "Users" (
	"UserID" INTEGER, 
	"FirstName" TEXT NOT NULL, 
	"LastName" TEXT NOT NULL, 
	"Email" TEXT NOT NULL, 
	"DateJoined" DATE NOT NULL, 
	PRIMARY KEY ("UserID"), 
	UNIQUE ("Email")
)

/*
3 rows from Users table:
UserID	FirstName	LastName	Email	DateJoined
1	Alice	Smith	alice.smith@email.com	2023-01-01
2	Bob	Johnson	bob.johnson@email.com	2023-02-15
3	Charlie	Brown	charlie.brown@email.com	2023-04-10
*/
Invoking: `sql_db_query` with `SELECT * FROM Users WHERE FirstName = 'Peter' LIMIT 10`


[(6, 'Peter', 'Johnson', 'peter.johnson@example.com', '2023-08-09')]Yes, we have a Peter in the database. Their information is as follows:
- First Name: Peter
- Last Name: Johnson
- Email: peter.johnson@example.com
- Date Joined: 2023-08-09

> Finished chain.


{'input': 'Do we have a Peter in the database?',
 'output': 'Yes, we have a Peter in the database. Their information is as follows:\n- First Name: Peter\n- Last Name: Johnson\n- Email: peter.johnson@example.com\n- Date Joined: 2023-08-09'}